In [20]:
import json
import data_cleaner
import pandas as pd

In [21]:
import os

# Specify the directory path
directory_path = 'data/'

# Get a list of files in the directory
file_names = [f for f in os.listdir(directory_path) if os.path.isfile(os.path.join(directory_path, f))]
file_names

['Alcatel_data.json',
 'Apple_data.json',
 'Asus_data.json',
 'Blu_data.json',
 'HTC_data.json',
 'Huawei_data.json',
 'Infinix_data.json',
 'Lenovo_data.json',
 'LG_data.json',
 'Nokia_data.json',
 'Samsung_data.json',
 'Sony_data.json',
 'Xiaomi_data.json',
 'ZTE_data.json']

In [22]:
dataframes = []
for f in file_names:
    file_path = directory_path + f
    with open(file_path, 'r') as file:
        data = json.load(file)
    parsed_data = []
    for brand, products in data.items():
        for product_name, spec in products.items():
            if "watch" in product_name.lower():
                continue
            lunch_data = data_cleaner.parse_date(spec.get('Launch', list(dict())))
            if not lunch_data[1]:
                continue
            display_information = data_cleaner.parse_display_information(spec.get("Display", list(dict())))
            memory_data = data_cleaner.parse_memory_data(spec.get("Memory", list(dict())))
            product_features = {
                "brand": brand,
                "product_name": product_name,
                "5g": data_cleaner.parse_5g(spec.get("Network", list(dict()))),
                "years_since_release": lunch_data[0],
                "current_status": lunch_data[1],
                "resolution": display_information[0],
                "size": display_information[1],
                "os": data_cleaner.os_parser(spec.get("Platform", list(dict()))),
                "battery": data_cleaner.get_battery(spec.get("Battery", list(dict()))),
                "ram": memory_data[0],
                "storage": memory_data[1],
                "price": data_cleaner.parse_price(spec.get("Misc", list(dict()))),
            }
            parsed_data.append(product_features)
    df = pd.DataFrame(parsed_data)
    dataframes.append(df)


ValueError: invalid literal for int() with base 10: '1440pi'

In [4]:
detailed_df = pd.concat(dataframes,ignore_index=True)
detailed_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5910 entries, 0 to 5909
Data columns (total 78 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   Category                    5910 non-null   object
 1   Product Name                5910 non-null   object
 2   Network_Technology          5910 non-null   object
 3   Network_2G bands            5910 non-null   object
 4   Network_3G bands            4328 non-null   object
 5   Network_4G bands            2648 non-null   object
 6   Network_Speed               4327 non-null   object
 7   Launch_Announced            5910 non-null   object
 8   Launch_Status               5910 non-null   object
 9   Body_Dimensions             5910 non-null   object
 10  Body_Weight                 5910 non-null   object
 11  Body_SIM                    5910 non-null   object
 12  Display_Type                5910 non-null   object
 13  Display_Size                5910 non-null   obje

In [5]:
detailed_df["Category"] = detailed_df["Category"].str.replace(r'\d+', '', regex=True)
detailed_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5910 entries, 0 to 5909
Data columns (total 78 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   Category                    5910 non-null   object
 1   Product Name                5910 non-null   object
 2   Network_Technology          5910 non-null   object
 3   Network_2G bands            5910 non-null   object
 4   Network_3G bands            4328 non-null   object
 5   Network_4G bands            2648 non-null   object
 6   Network_Speed               4327 non-null   object
 7   Launch_Announced            5910 non-null   object
 8   Launch_Status               5910 non-null   object
 9   Body_Dimensions             5910 non-null   object
 10  Body_Weight                 5910 non-null   object
 11  Body_SIM                    5910 non-null   object
 12  Display_Type                5910 non-null   object
 13  Display_Size                5910 non-null   obje

In [6]:
detailed_df.isnull().sum()

Category                         0
Product Name                     0
Network_Technology               0
Network_2G bands                 0
Network_3G bands              1582
                              ... 
Tests_Battery (new)           5863
Selfie camera_Triple          5904
Main Camera_Dual or Triple    5908
Main Camera_Penta             5909
Main Camera_Five              5907
Length: 78, dtype: int64

In [7]:
detailed_df.drop(["Network_2G bands","Network_3G bands","Network_4G bands"],axis=1,inplace=True)

In [8]:
detailed_df

,Category,Product Name,Network_Technology,Network_Speed,Launch_Announced,Launch_Status,Body_Dimensions,Body_Weight,Body_SIM,Display_Type,...,Features_Clock,Features_Alarm,Features_Languages,Network_5G bands,Tests_Performance,Tests_Battery (new),Selfie camera_Triple,Main Camera_Dual or Triple,Main Camera_Penta,Main Camera_Five
0,alcatel,1B (2022),GSM / HSPA / LTE,"HSPA 42.2/11.1 Mbps, LTE Cat4 150/50 Mbps","2022, May 31","Available. Released 2022, May 31",146.7 x 71.9 x 10 mm (5.78 x 2.83 x 0.39 in),172 g (6.07 oz),"Single SIM (Nano-SIM) or Dual SIM (Nano-SIM, d...",TFT LCD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,alcatel,1L Pro (2021),GSM / HSPA / LTE,"HSPA 21.1/5.76 Mbps, LTE Cat4 150/50 Mbps","2021, June 25","Available. Released 2021, Q2",156.4 x 74.8 x 9.7 mm (6.16 x 2.94 x 0.38 in),190 g (6.70 oz),Single SIM (Nano-SIM) or Hybrid Dual SIM (Nano...,IPS LCD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,alcatel,1 (2021),GSM / HSPA / LTE,"HSPA 42.2/11.5 Mbps, LTE Cat4 150/50 Mbps","2021, June 25","Available. Released 2021, October 22",137.6 x 65.7 x 9.8 mm (5.42 x 2.59 x 0.39 in),134 g (4.73 oz),"Single SIM (Nano-SIM) or Dual SIM (Nano-SIM, d...",TFT LCD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,alcatel,3L (2021),GSM / HSPA / LTE,"HSPA 42.2/5.76 Mbps, LTE Cat4 150/50 Mbps","2021, January 12","Available. Released 2021, April 26",165.6 x 75.6 x 8.7 mm (6.52 x 2.98 x 0.34 in),194 g (6.84 oz),"Single SIM (Nano-SIM) or Dual SIM (Nano-SIM, d...",IPS LCD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,alcatel,1S (2021),GSM / HSPA / LTE,"HSPA 42.2/5.76 Mbps, LTE Cat4 150/50 Mbps","2021, January 12","Available. Released 2021, March 25",165.6 x 75.6 x 8.8 mm (6.52 x 2.98 x 0.35 in),190 g (6.70 oz),Single SIM (Nano-SIM) or Hybrid Dual SIM (Nano...,IPS LCD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5905,ZTE,F101,GSM / UMTS,"Yes, 384 kbps",2009. Released 2009,Discontinued,105 x 46 x 12.6 mm (4.13 x 1.81 x 0.50 in),90 g (3.17 oz),Mini-SIM,"TFT, 65K colors",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5906,ZTE,F100,GSM / UMTS,"Yes, 384 kbps",2009. Released 2009,Discontinued,105 x 46 x 12.6 mm (4.13 x 1.81 x 0.50 in),90 g (3.17 oz),Mini-SIM,"TFT, 65K colors",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5907,ZTE,Coral200 Sollar,GSM,NaN,"2007, May. Released 2007",Discontinued,106 x 45.6 x 18.1 mm (4.17 x 1.80 x 0.71 in),85 g (3.00 oz),Mini-SIM,"CSTN, 65K colors",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5908,ZTE,Blade V20,GSM / HSPA / LTE,"HSPA, LTE-A",Not announced yet,Cancelled,-,-,"Dual SIM (Nano-SIM, dual stand-by)",IPS LCD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
